## Differential ligands + expressed receptors

In [ ]:
import os
import sys
print("Python version" + sys.version)
os.getcwd()
print(sys.executable)

In [2]:
import numpy as np
np.random.seed(123)
import pandas as pd
import scipy
import itertools

import umap
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import scanpy as sc
import anndata as ad
import scvelo as scv
from tqdm.notebook import tqdm

from pathlib import Path

In [3]:
import scanpy.external as sce

In [ ]:
sc.settings.verbosity = 1
sc.logging.print_header()
sc.settings.set_figure_params(dpi=300, facecolor='white')

In [5]:
# remove weird grid from scvelo
plt.rcParams['axes.grid'] = False
plt.rcParams['figure.dpi'] = 150

In [6]:
# revised from Stefan's cell type signature
signatures_path_ = '../cell_type_from_stefan/scrnaseq_signature_collection/'
from score_and_classify import *

In [7]:
data_folder = '../data/'

In [8]:
new_data_folder = '../processed_data/'

In [9]:
adata_all = sc.read(Path(new_data_folder)/'adata_all_full_cell_type_annotation.h5')

In [10]:
adata_epi = sc.read(Path(new_data_folder)/'CB_epi_Numbat_Scitcem_inferCNV_icms_Uhlitz_resolved_identity.h5')

In [11]:
adata_epi.uns['tumour_normal_normal_colors'] = ['#ff7f0e', '#9b1ee3','#1f77b4','#808080']

In [12]:
cell_type_list = adata_epi.obs['Uhlitz_scANVI'].cat.categories.values

In [13]:
cell_type_order = ['TC1', 'TC2', 'TC3', 'TC4', 'Stem', 'Stem/TA',
                    'Tuft', 'Immature Goblet', 'Goblet', 'Enterocyte progenitor', 'Enterocytes']

In [14]:
adata_epi.obs['Uhlitz_scANVI'] = adata_epi.obs['Uhlitz_scANVI'].cat.reorder_categories(cell_type_order)

In [15]:
adata_epi.uns['Uhlitz_scANVI_colors'] = ['#AA0000', '#FF0000', '#FF0099', '#EB9999','#00CCFF', '#0072b1','#FF9900', 
 '#990099','#660099', '#999900', '#336600']

In [16]:
sc.tl.diffmap(adata_epi, n_comps=15)

In [17]:
adata_epi_reindex = adata_epi.obs.reindex(adata_all.obs.index)

In [18]:
adata_all.obs['cell_identity'] = None

for i in np.arange(0, adata_all.shape[0]):
    if adata_epi_reindex['cell_identity'][i] == 'normal_sample':
        adata_all.obs['cell_identity'][i] = 'normal_sample'
    elif adata_epi_reindex['cell_identity'][i] == 'genomically_normal':
        adata_all.obs['cell_identity'][i] = 'genomically_normal'
    elif adata_epi_reindex['cell_identity'][i] == 'genomically_tumour':
        adata_all.obs['cell_identity'][i] = 'genomically_tumour'
    elif adata_epi_reindex['cell_identity'][i] == 'no confident assignment':
        adata_all.obs['cell_identity'][i] = 'no confident assignment'
    else:
        adata_all.obs['cell_identity'][i] = adata_all.obs['cell_type_level2'][i]
        
adata_all.obs['cell_identity'] = adata_all.obs['cell_identity'].astype('category')


#### TC filtering 

In [19]:
# fix TCs
# genomically tumour = Tumour like
adata_epi.obs['Uhlitz_scANVI_noTC'] = None

for i in np.arange(0, adata_epi.shape[0]):
    if adata_epi.obs['cell_identity'][i] == 'genomically_tumour':
        adata_epi.obs['Uhlitz_scANVI_noTC'][i] = 'Tumour-like'        
    elif 'TC' in adata_epi.obs['Uhlitz_scANVI'][i]:
        adata_epi.obs['Uhlitz_scANVI_noTC'][i] = 'Tumour-like (normal_sample)'        
    else: 
        adata_epi.obs['Uhlitz_scANVI_noTC'][i] = adata_epi.obs['Uhlitz_scANVI'][i]
        
adata_epi.obs['Uhlitz_scANVI_noTC'] = adata_epi.obs['Uhlitz_scANVI_noTC'].astype('category')

In [20]:
adata_epi.obs['Uhlitz_scANVI_noTC'] = adata_epi.obs['Uhlitz_scANVI_noTC'].cat.reorder_categories(['Tumour-like', 
                                                                                                  'Tumour-like (normal_sample)',
                                                                                                  'Stem', 'Stem/TA', 
                                                           'Tuft',
                                                           'Immature Goblet', 'Goblet',
                                                           'Enterocyte progenitor', 'Enterocytes'])

In [21]:
Uhlitz_scANVI_noTC_color_list = ['#AA0000', '#808080',
                                              '#00CCFF', '#0072b1',
                                              '#FF9900',
                                              '#990099', '#660099',
                                              '#999900', '#336600']

adata_epi.uns['Uhlitz_scANVI_noTC_colors'] = Uhlitz_scANVI_noTC_color_list

In [22]:
Uhlitz_scANVI_noTC_color_list_after = ['#00CCFF', '#0072b1',
                                              '#FF9900',
                                              '#990099', '#660099',
                                              '#999900', '#336600']

In [23]:
adata_epi_noTC_cleanCI = adata_epi[(adata_epi.obs['Uhlitz_scANVI_noTC'] != 'Tumour-like (normal_sample)') & #37670
                                   (adata_epi.obs['cell_identity'] != 'no confident assignment')] # 37316

In [24]:
sample_color_map = dict(zip(adata_epi_noTC_cleanCI.obs['Uhlitz_scANVI_noTC'].cat.categories, 
                            adata_epi_noTC_cleanCI.uns['Uhlitz_scANVI_noTC_colors']))

In [25]:
removed_TC_id = list(set(adata_epi.obs.index) - set(adata_epi_noTC_cleanCI.obs.index))

In [26]:
adata_all_noTC_cleanCI = adata_all[~adata_all.obs.index.isin(removed_TC_id)]

In [28]:
sample_color_map_for_all = sample_color_map

sample_color_map_for_all.update(dict(zip(adata_all_noTC_cleanCI[adata_all_noTC_cleanCI.obs['celltype_1a'] != 'epi'].obs['cell_type_level2'].cat.categories, 
         ['#7d8df0', '#335B8E', '#79c9c9', '#67C9FF',
          '#CE9486', '#8f7022', '#cf7708', '#B5B867',
          '#DEB058', '#899DA4','#1aa375', '#AA82A7', '#c44545'
         ])))

In [29]:
adata_all_noTC_cleanCI.uns['cell_type_level2_colors'] = ['#7d8df0', '#335B8E', '#79c9c9', '#67C9FF',
                                                         '#CE9486', '#999900', '#336600', 
                                                         '#8f7022', '#cf7708', 
                                                         '#B5B867','#660099', '#990099',
                                                         '#DEB058', '#899DA4','#1aa375', '#AA82A7',  '#00CCFF', '#0072b1',
                                                         '#AA0000', '#FF0000', '#FF0099', '#EB9999',
                                                         '#c44545', '#FF9900'
]

### gene expression

In [ ]:
# clock genes
sc.pl.dotplot(adata_all_noTC_cleanCI, 
              var_names=['ARNTL','PER3', 'RORC'], groupby = 'cell_type_level2', swap_axes=True)

### pathway

In [ ]:
CRC_sig = pd.read_excel("../datasets_new_preprocessing/Signatures_Single_cells.xlsx", 
                        skiprows=[1], index_col=None, sheet_name = 'fixed_var_name')
for i in CRC_sig.columns:
    score_genes(adata_epi_noTC_cleanCI, np.array(CRC_sig[i].dropna()), score_name=i)

In [ ]:
with plt.style.context('./plt_style'):
    ax1 = sc.pl.heatmap(adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] =='genomically_tumour')], 
                        CRC_sig.columns, groupby='Uhlitz_scANVI', 
                        dendrogram=False, swap_axes=True, figsize=[12,6], 
                        vcenter= 0,
                        cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True), 
                        show = False
                  #row_colors = adata_epi[(adata_epi.obs['cell_identity'] =='genomically_normal')].obs['Uhlitz_scANVI'].sort_values().map(sample_color_map)
                 )
    ax1['heatmap_ax'].set_title('CRC signatures activity in genomically tumour cells', fontsize = 16)
    ax1['groupby_ax'].set_xlabel('')


#AttributeError: 'AxesImage' object has no property 'row_colors'

In [ ]:
with plt.style.context('./plt_style'):
    ax1 = sc.pl.heatmap(adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] =='genomically_tumour')], 
                        CRC_sig.columns, groupby='Uhlitz_scANVI_noTC', 
                        dendrogram=False, swap_axes=True, figsize=[12,6], 
                        vcenter= 0,
                        cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True), 
                        show = False
                  #row_colors = adata_epi[(adata_epi.obs['cell_identity'] =='genomically_normal')].obs['Uhlitz_scANVI'].sort_values().map(sample_color_map)
                 )
    ax1['heatmap_ax'].set_title('CRC signatures activity in genomically tumour cells', fontsize = 16)


#AttributeError: 'AxesImage' object has no property 'row_colors'

In [ ]:
with plt.style.context('./plt_style'):
    ax2 = sc.pl.heatmap(adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] =='genomically_normal')], 
                        CRC_sig.columns, groupby='Uhlitz_scANVI_noTC', 
                        dendrogram=False, swap_axes=True, figsize=[12,6], 
                        vcenter= 0,
                        cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),
                        show = False
                  #row_colors = adata_epi[(adata_epi.obs['cell_identity'] =='genomically_normal')].obs['Uhlitz_scANVI'].sort_values().map(sample_color_map)
                 )
    #ax2['heatmap_ax'].set_title('CRC signatures activity in genomically normal cells', fontsize = 16)
    ax2['groupby_ax'].set_xlabel('')

    #ax2['groupby_ax'].plot(c = ['red']*7)
    #print(ax2)
#AttributeError: 'AxesImage' object has no property 'row_colors'

In [ ]:
with plt.style.context('./plt_style'):
    ax3 = sc.pl.heatmap(adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] =='normal_sample')], 
                  CRC_sig.columns, groupby='Uhlitz_scANVI_noTC', 
                   dendrogram=False, swap_axes=True, figsize=[12,6], 
                  vcenter= 0,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),show = False
                  #row_colors = adata_epi[(adata_epi.obs['cell_identity'] =='genomically_normal')].obs['Uhlitz_scANVI'].sort_values().map(sample_color_map)
                 )
    #ax3['heatmap_ax'].set_title('CRC signatures activity in normal sample', fontsize = 16)
    ax3['groupby_ax'].set_xlabel('')

#AttributeError: 'AxesImage' object has no property 'row_colors'

In [ ]:
with plt.style.context('./plt_style'):
    ax4 = sc.pl.heatmap(adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] =='normal_sample')], 
                  CRC_sig.columns, groupby='louvain', 
                   dendrogram=False, swap_axes=True, figsize=[12,6], 
                  vcenter= 0,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),show = False
                  #row_colors = adata_epi[(adata_epi.obs['cell_identity'] =='genomically_normal')].obs['Uhlitz_scANVI'].sort_values().map(sample_color_map)
                 )
    #ax3['heatmap_ax'].set_title('CRC signatures activity in normal sample', fontsize = 16)
    ax4['groupby_ax'].set_xlabel('')

#AttributeError: 'AxesImage' object has no property 'row_colors'

In [ ]:
with plt.style.context('./plt_style'):
    ax5 = sc.pl.heatmap(adata_epi_noTC_cleanCI[(adata_epi_noTC_cleanCI.obs['cell_identity'] =='genomically_normal')], 
                  CRC_sig.columns, groupby='louvain', 
                   dendrogram=False, swap_axes=True, figsize=[12,6], 
                  vcenter= 0,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),show = False
                  #row_colors = adata_epi[(adata_epi.obs['cell_identity'] =='genomically_normal')].obs['Uhlitz_scANVI'].sort_values().map(sample_color_map)
                 )
    #ax3['heatmap_ax'].set_title('CRC signatures activity in normal sample', fontsize = 16)
    ax5['groupby_ax'].set_xlabel('')

#AttributeError: 'AxesImage' object has no property 'row_colors'

In [41]:
pathway_df = pd.melt(adata_epi_noTC_cleanCI.obs[['cell_identity', 'Uhlitz_scANVI_noTC',
                                     'TA-Merlos', 'Enterocyte-Merlos', 'Paneth-Sato',
       'NFkB_targets-Schwitalla', 'BMP_targets_up-Qi', 'Lgr5_ISC-Merlos',
       'EphB2_ISC-Merlos', 'Late_TA-Merlos', 'Lgr5_ISC-Munoz', 'WYR-Serra',
       'EpiHR-Canellas', 'YAP22_Wang', 'RevivalStemCell-Ayyaz',
       'Fetal_Spheroid_mustata', 'Ki67hi_Proliferative-Cortina',
       'MEX3A-Barriga', 'MEX3Ahi_LGR5hi-Alvarez', 'MEX3Alo_LGR5hi-Alvarez']].reset_index(),
       id_vars=['index', 'cell_identity', 'Uhlitz_scANVI_noTC'], value_name='score', var_name='pathway')

In [42]:
pathway_df['cell_identity'] = pathway_df['cell_identity'].cat.reorder_categories(['normal_sample', 
                                                                                  'genomically_normal', 
                                                                                  'genomically_tumour'])

In [45]:
#revise from https://stackoverflow.com/questions/41471238/how-to-make-heatmap-square-in-seaborn-facetgrid
def plot_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.pivot_table(index=args[0], columns=args[1], values=args[2], aggfunc = 'median')
    sns.heatmap(d, **kwargs)

In [ ]:
with plt.style.context('./plt_style'):
    g = sns.FacetGrid(pathway_df,
                      col = 'Uhlitz_scANVI_noTC', 
                      col_order = [#'TC1', 'TC2', 'TC3', 'TC4', 
                          'Tumour-like','Stem', 'Stem/TA', 
                          'Tuft', 'Immature Goblet', 'Goblet', 'Enterocyte progenitor', 'Enterocytes'],
                      height = 12, aspect = 2/9,
                      despine = False, sharey = True)
    
    cbar_ax = g.fig.add_axes([.92, .3, .02, .4])
    
    g.map_dataframe(plot_heatmap, 'pathway', 'cell_identity', 'score', cbar_ax=cbar_ax, 
                    cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),
                    center = 0
                    #, vmin=-0.1, vmax=0.4
                   )
    g.figure.subplots_adjust(wspace=0.1)
    g.set_titles('{col_name}\n')
    g.set_xlabels('')
    g.set_ylabels('')
    g.set(xticks = np.arange(0,2,0.5))
    g.set_xticklabels(['','normal sample','','tumour sample'], rotation = 90)
    # Tumour-like fix manually

    axes = g.axes.flatten()
    axes[6].set_title('Enterocyte\nprogenitor\n')
    axes[7].set_title('Enterocyte\n')
    axes[4].set_title('Immature\ngoblet\n')
    
    g.fig.subplots_adjust(right=.9)
    

In [47]:
import decoupler as dc

In [48]:
progeny = dc.get_progeny(organism='human', top=500)

In [ ]:
progeny['source'].unique()

In [ ]:
dc.run_mlm(
    mat=adata_epi_noTC_cleanCI,
    net=progeny,
    source='source',
    target='target',
    weight='weight',
    verbose=True,
    use_raw=False
)

In [52]:
acts = dc.get_acts(adata_epi_noTC_cleanCI, obsm_key='mlm_estimate')

In [ ]:
with plt.style.context('./plt_style'):
    ax1 = sc.pl.heatmap(acts[(acts.obs['cell_identity'] =='genomically_tumour')], 
                  acts.var_names, groupby='Uhlitz_scANVI', 
                  dendrogram=False, swap_axes=True, figsize=[12,4], vmin = -45, vmax = 45, 
                  vcenter= 0, show = False,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),)
    ax1['heatmap_ax'].set_title('PROGENy in genomically tumour', fontsize = 16)
    ax1['groupby_ax'].set_xlabel('')
    
with plt.style.context('./plt_style'):
    ax2 = sc.pl.heatmap(acts[(acts.obs['cell_identity'] =='genomically_tumour')], 
                  acts.var_names, groupby='Uhlitz_scANVI_noTC', 
                  dendrogram=False, swap_axes=True, figsize=[12,4], vmin = -45, vmax = 45, 
                  vcenter= 0,show = False,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),)
    ax2['heatmap_ax'].set_title('PROGENy in genomically tumour', fontsize = 16)


with plt.style.context('./plt_style'):
    ax3 = sc.pl.heatmap(acts[(acts.obs['cell_identity'] =='genomically_normal')], 
                  acts.var_names, groupby='Uhlitz_scANVI_noTC', 
                  dendrogram=False, swap_axes=True, figsize=[12,4], vmin = -45, vmax = 45,
                  vcenter= 0,show = False,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),)
    ax3['heatmap_ax'].set_title('PROGENy in genomically normal', fontsize = 16)
    ax3['groupby_ax'].set_xlabel('')
    
with plt.style.context('./plt_style'):
    ax4 = sc.pl.heatmap(acts[acts.obs['cell_identity'] =='normal_sample'], 
                  acts.var_names, groupby='Uhlitz_scANVI_noTC', 
                  dendrogram=False, swap_axes=True, figsize=[12,4], vmin = -45, vmax = 45,
                  vcenter= 0,show = False,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),)    
    ax4['heatmap_ax'].set_title('PROGENy in normal sample', fontsize = 16)
    ax4['groupby_ax'].set_xlabel('')
;    

In [ ]:
with plt.style.context('./plt_style'):
    ax1 = sc.pl.heatmap(acts[(acts.obs['cell_identity'] =='genomically_normal')], 
                  acts.var_names, groupby='louvain', 
                  dendrogram=False, swap_axes=True, figsize=[12,4], vmin = -45, vmax = 45, 
                  vcenter= 0, show = False,
                  cmap = sns.color_palette("blend:#0000CD,#D9D9D9,#CD0000", as_cmap=True),)
    ax1['heatmap_ax'].set_title('PROGENy in cluster 9 & genomically normal', fontsize = 16)
    ax1['groupby_ax'].set_xlabel('')

#### PROGENy on all cells

In [ ]:
dc.run_mlm(
    mat=adata_all_noTC_cleanCI,
    net=progeny,
    source='source',
    target='target',
    weight='weight',
    verbose=True,
    use_raw=False)

####be careful of the index, must be str/cell_id index, or the Anndata pkg will throw an error when adding to obsm

In [56]:
acts_all = dc.get_acts(adata_all_noTC_cleanCI, obsm_key='mlm_estimate')

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.heatmap(acts_all, acts_all.var_names, groupby='cell_type_level2', 
                  dendrogram=False, swap_axes=True, figsize=[20,4])
;    

### ICR genes from Nature Medicine | Volume 29 | May 2023 | 1273–1286

In [67]:
ICR_genes = {'Th1':['IFNG', 'TBX21', 'CD8A', 'CD8B', 'IL12B', 'STAT1', 'IRF1'], # Th1 cell signalling
            'chemokines':['CXCL9', 'CXCL10', 'CCL5'], # CXCR3/CCR5 chemokines
            'effectors':['GNLY', 'PRF1', 'GZMA', 'GZMB', 'GZMH'], # effector functions
            'immune regulatory':['CD274', 'CTLA4', 'FOXP3', 'IDO1', 'PDCD1'] # immune regulatory
            }

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, ICR_genes, groupby=['cell_type_level2'], dendrogram=False)

### LR gene expression

In [69]:
new_order = ['TC1', 'TC2', 'TC3', 'TC4', 'Stem', 'Stem/TA',
             'Immature Goblet', 'Goblet',
             'Enterocyte progenitor', 'Enterocytes', 'Tuft']

#### WNT

In [70]:
WNT_ligand=['WNT1','WNT2','WNT2B','WNT3','WNT3A','WNT4',
                          'WNT5A','WNT5B','WNT6','WNT7A','WNT7B',#'WNT8A',
                          'WNT8B','WNT9A','WNT9B','WNT10A','WNT10B','WNT11','WNT16', 
            'RSPO1','RSPO2','RSPO3','RSPO4'] # WNT amp
WNT_receptor_cofactor=['FZD1','FZD2','FZD3','FZD4','FZD5','FZD6','FZD7','FZD8','FZD9','FZD10',
                        'LGR4','LGR5','LGR6'] # WNT amp


In [71]:
WNT_list = {'WNT_ligand':WNT_ligand,
           'WNT_receptor_cofactor':WNT_receptor_cofactor}

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, WNT_receptor_cofactor, 
                  groupby=['sample_origin','cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, WNT_receptor_cofactor, groupby=['cell_identity'], dendrogram=False)
;    

In [ ]:
# WNT receptors in Epi
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, WNT_list['WNT_receptor_cofactor'], 
                  groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False, swap_axes=True)
;    

In [ ]:
# WNT receptors in Epi
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, WNT_list, 
                  groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False, swap_axes=True)
;    

In [ ]:
# PROGENy pathway activity, only FZD9 is shared between the LR list and PROGENy targets
with plt.style.context('./plt_style'):
    sc.pl.dotplot(acts, 'WNT', groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False)
;  


In [ ]:
scv.pl.scatter(acts, basis='umap', color='WNT',
               ncols=1, dpi=150, legend_loc='right margin', size = 2)

In [ ]:
scv.pl.scatter(adata_epi_noTC_cleanCI, basis='umap', color=['WNT2B','FZD5', 'BMP4'],
               ncols=3, dpi=150, legend_loc='right margin', size = 2)

In [ ]:
# stem markers
scv.pl.scatter(adata_epi_noTC_cleanCI, basis='umap', color=['LGR5','OLFM4','EPHB2', 'XRCC2', 'MMP7',
                                                           'ASCL2', 'SOX9', 'TNFRSF19', 'BCL2',
                                                           'SOX4', 'CD44', 'YAP1', 'KIT', 'JUN', 'MIF', 'NPM1'],
               ncols=3, dpi=150, legend_loc='right margin', size = 2,
               vmin = 0, vmax = 5)

In [ ]:
# c9 markers
scv.pl.scatter(adata_epi_noTC_cleanCI, basis='umap', color=['PLA2G2A', 'PIGR', 'LEFTY1', 
                                                            'B3GNT7', 'SI','SPINK5', 
                                                            'MUC12','LCN2', 'DMBT1',
                                                            'OLFM4', 'DUOX2','PRAC1', 
                                                            'PI3', 'SLPI', 'B3GALT5', 
                                                            'GOLM1','NXPE4', 'NXPE1'],
               ncols=6, dpi=150, legend_loc='right margin', size = 2,
              color_map='jet', vmin = 0, vmax = 5)

In [ ]:
# c9 markers
scv.pl.scatter(adata_epi_noTC_cleanCI, basis='umap', color=['PLA2G2A', 'PIGR', 'LEFTY1', 
                                                            'B3GNT7', 'MUC12','LCN2', 
                                                            'OLFM4', 'PI3', 'SLPI', 
                                                            'GOLM1','NUPR1', 'MUC1',
                                                           'MUC5B', 'SELENOP', 'ZG16'],
               ncols=5, dpi=150, legend_loc='right margin', size = 2,
              color_map='jet', vmin = 0, vmax = 5)

In [ ]:
# playaround
scv.pl.scatter(adata_epi_noTC_cleanCI, basis='umap', color=['BEST4'],
               ncols=6, dpi=150, legend_loc='right margin', size = 2,
              color_map='jet', vmin = 0, vmax = 5)

In [ ]:
# stem markers
scv.pl.scatter(adata_epi_noTC_cleanCI, basis='umap', color=['LGR5','OLFM4','EPHB2', 'XRCC2', 'MMP7',
                                                           'ASCL2', 'SOX9', 'TNFRSF19', 'BCL2',
                                                           'SOX4', 'CD44', 'YAP1', 'KIT', 'JUN', 'MIF', 'NPM1'],
               ncols=3, dpi=150, legend_loc='right margin', size = 2,
              color_map='jet', vmin = 0, vmax = 5)

In [ ]:
scv.pl.scatter(adata_all_noTC_cleanCI, basis='umap', color=['WNT2B','FZD5', 'BMP4','FOXP3', 'JUNB'],
               ncols=3, dpi=150, legend_loc='right margin', size = 2)

#### EGF

In [85]:
EGF_ligand = ['EGF','TGFA','HBEGF','BTC','AREG','EREG','EPGN']
EGF_receptor = ['EGFR','ERBB2','ERBB3','ERBB4']

EGF_list = {'EGF_ligand':['EGF','TGFA','HBEGF','BTC','AREG','EREG','EPGN'],
            'EGF_receptor':['EGFR','ERBB2','ERBB3','ERBB4']
}

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, EGF_receptor, groupby=['sample_origin','cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, EGF_receptor, groupby=['cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, EGF_receptor, groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False,
                 swap_axes=True)
; 

#### FGF

In [89]:
FGF_ligand = ['FGF1','FGF2','FGF3','FGF4','FGF5',#'FGF6',
              'FGF7','FGF8','FGF9','FGF10']
FGF_receptor = ['FGFR1','FGFR2','FGFR3','FGFR4']

FGF_list = {
    'FGF_ligand':['FGF1','FGF2','FGF3','FGF4','FGF5',#'FGF6',
                  'FGF7','FGF8','FGF9','FGF10'],
    'FGF_receptor':['FGFR1','FGFR2','FGFR3','FGFR4']
}

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, FGF_receptor, groupby=['sample_origin','cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, FGF_receptor, groupby=['cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, FGF_receptor, groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False,
                 swap_axes=True)
; 

#### BMP

In [93]:
BMP_ligand = ['BMP2','BMP4','BMP6','BMP7']
BMP_receptor = ['BMPR1A','BMPR1B','BMPR2']

BMP_list = {
    'BMP_ligand':['BMP2','BMP4','BMP6','BMP7'],
    'BMP_receptor':['BMPR1A','BMPR1B','BMPR2']

}

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, BMP_receptor, groupby=['sample_origin','cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, BMP_receptor, groupby=['cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, BMP_receptor, groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False,
                 swap_axes=True)
; 

#### TGFB

In [97]:
TGFB_ligand = ['TGFB1','TGFB2','TGFB3','LTBP1','LRRC32','BMP3',#'GDF2','BMP10',
               'INHA','INHBA','INHBB',
               'INHBC','INHBE','FST']
TGFB_receptor = ['TGFBR1','TGFBR2','ACVR1','ACVR1B','ACVR1C',#'AVCR2A',
                 'ACVR2B']

TGFB_list = {'TGFB_ligand':TGFB_ligand,
             'TGFB_receptor':TGFB_receptor
    
}

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, TGFB_receptor, groupby=['sample_origin','cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, TGFB_receptor, groupby=['cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, TGFB_receptor, groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False,
                 swap_axes=True)
; 

#### NOTCH

In [101]:
NOTCH_ligand = ['JAG1','JAG2','DLL1','DLL3','DLL4']
NOTCH_receptor = ['NOTCH1','NOTCH2','NOTCH3','NOTCH4']

NOTCH_list = {'NOTCH_ligand':NOTCH_ligand,
             'NOTCH_receptor':NOTCH_receptor}

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, NOTCH_receptor, groupby=['sample_origin','cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, NOTCH_receptor, groupby=['cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, NOTCH_receptor, groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False,
                 swap_axes=True)
; 

#### EPHRIN

In [105]:
EPH_ligand = ['EFNA1','EFNA2','EFNA3','EFNA4','EFNA5','EFNB1','EFNB2','EFNB3']
EPH_receptor = ['EPHA1','EPHA2','EPHA3','EPHA4',
                'EPHA5','EPHA6','EPHA7','EPHA8',
                'EPHB1','EPHB2','EPHB3','EPHB4',#'EPHB5',
                'EPHB6']

EPH_list = {
    'EPH_ligand':EPH_ligand,
    'EPH_receptor':EPH_receptor
}

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, EPH_list, groupby=['sample_origin','cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, EPH_list, groupby=['cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, EPH_list, groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False,
                 swap_axes=True)
; 

#### Hippo/YAP

In [109]:
HIP_ligand = ['DCHS1', 'DCHS2']
HIP_receptor = ['FAT1', 'FAT2', 'FAT3', 'FAT4']

HIP_list = {
    'HIP_ligand':HIP_ligand,
    'HIP_receptor':HIP_receptor
}

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_all_noTC_cleanCI, HIP_list, groupby=['sample_origin','cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, HIP_list, groupby=['cell_identity'], dendrogram=False)
;    

In [ ]:
with plt.style.context('./plt_style'):
    sc.pl.dotplot(adata_epi_noTC_cleanCI, HIP_list, groupby=['Uhlitz_scANVI_noTC','cell_identity'], dendrogram=False,
                 swap_axes=True)
; 